In [8]:
import os
import cv2
import sys
import pytesseract 
from PIL import Image
import re
import subprocess
import sqlite3  

PATH_PDFTOTEXT = r"D:\New\ML Group\xpdf-tools-win-4.00\xpdf-tools-win-4.00\bin32\pdftotext";
dbname = r"D:\New\ML Group\ProjImageAndPDF\db.sqlite3"

def tagging():
    global dbname
    if os.path.exists(dbname):
        os.remove(dbname)
    createdb()    
    #ask dir
    rootdir = r'D:\New\ML Group\ProjImageAndPDF\docs'
    r2 = input('please enter dir to search')
    if r2 != None and r2 != "":
        rootdir = r2
    
    file_list = []
    #loop through dir
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            file_list.append(os.path.join(subdir, file))

    #loop throw files:
    for f1 in file_list:
        if f1.endswith(r"\tmp.png"):
            continue
        if f1.endswith(".png") or f1.endswith(".jpg") or f1.endswith(".jpeg"):
#                 print(f1)
                pan, yr = tagFromImg(f1)
                tags = "{},{}".format(pan, yr)
                insertdb(tags, f1)
#                 print("="*40)
#     print("="*50)
    for f1 in file_list:
        if f1.endswith(".pdf"):
#                 print(f1)
                pan, yr = tagFromPDF(f1)
                insertdb(tags, f1)
    print("Done!")
def tagFromImg(fullPath):
  src_path, name1 = os.path.split(fullPath) 
  src_path = src_path + "\\"
#   src_path = os.path.dirname(fullPath)
  imPath = src_path + name1
  img = Image.open(imPath)

  x1, y1, x2, y2 = 1200,300,1500,400
  crop_img = img.crop((x1, y1, x2, y2))
  crop_img.save(src_path+'tmp.png')
    
  imPath = src_path + "tmp.png"
  config = ('-l eng --oem 1 --psm 6')
  # Read image from disk
  im = cv2.imread(imPath, cv2.IMREAD_COLOR) 
  # Run tesseract OCR on image
  text = pytesseract.image_to_string(im, config=config)

  pat1 = "[A-Z]{5}\d{4}[A-Z]"
  pan = re.search(pat1, text).group()
#   print(pan)

  imPath = src_path + name1
  img = Image.open(imPath)
  x1, y1, x2, y2 = 1300,180,1900,350
  crop_img = img.crop((x1, y1, x2, y2))
  crop_img.save(src_path+'tmp.png')
  imPath = src_path + "tmp.png"
     
  config = ('-l eng --oem 1 --psm 6')
  # Read image from disk
  im = cv2.imread(imPath, cv2.IMREAD_COLOR)
  # Run tesseract OCR on image
  text = pytesseract.image_to_string(im, config=config)
  pat1 = "\d{4}[ ]*-[ ]*\d{2}"
  yr = re.search(pat1, text).group()
#   print(yr)
  return pan, yr

def tagFromPDF(fn):
    global PATH_PDFTOTEXT
    src_path, name1 = os.path.split(fn) 
    args = [r"{}".format(PATH_PDFTOTEXT),
            '-enc',
            'UTF-8',
            "{}".format(fn),
            '-']
    res = subprocess.run(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output = res.stdout.decode('utf-8')
    pat1 = "[A-Z]{5}\d{4}[A-Z]"
    pan = re.search(pat1, output).group()
#     print(pan)
    pat2 = "\d{4}[ ]*-[ ]*\d{2}"
    yr = re.search(pat2, output).group()
#     print(yr)    
    return pan, yr

def createdb():
    global dbname
    con = sqlite3.connect(dbname)
    cur = con.cursor() # instantiate a cursor obj
    create_sql = """
     CREATE TABLE IF NOT EXISTS tags (
         id integer PRIMARY KEY,
         tags text NOT NULL,
         file_name text NOT NULL)"""
    cur.execute(create_sql)
    con.commit()

def insertdb(tags, fn):
    global dbname
    con = sqlite3.connect(dbname)
    cur = con.cursor() # instantiate a cursor obj
    create_sql = """
    INSERT INTO tags (tags, file_name) VALUES (?, ?)"""
    cur.execute(create_sql, (tags, fn))
    con.commit()
#     print(tags)
#     print(fn)

def selectdb(tags):
    global dbname
    con = sqlite3.connect(dbname)
    cur = con.cursor() # instantiate a cursor obj
    create_sql = """
    select * from tags """
    c = 0 
    for t1  in tags:
        if c ==0:
            create_sql = create_sql + " where tags like '{}' ".format("%"+t1.strip()+"%")
        else:
            create_sql = create_sql + " and tags like '{}' ".format("%"+t1.strip()+"%")
        c=c+1    
    cur.execute(create_sql)
    results = cur.fetchall()
    con.commit()
    for row in results:
        print(row)
    return results
#     print(tags)
#     print(fn)

def search():    
    tags = input('Please enter tag to search')
    tags = tags.split(",")
    selectdb(tags)
    
    
if __name__ == "__main__":
    print("to tag enter t")
    print("to search enter s")
    ch = input('your choice\n')
    if ch.lower() == 't': 
        tagging()
    else:
        search()

to tag enter t
to search enter s
your choice
t
please enter dir to search


AttributeError: 'NoneType' object has no attribute 'group'